In [1]:
import smtplib
import keyring
import sqlite3
import pandas as pd
from email.message import EmailMessage
from datetime import date
import csv

In [13]:
# Games id that I'm interested in buying with a discount price
games_id = []

# Desired price
price_wanted = 17

with open('games.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        games_id.append(row[0])

con = sqlite3.connect(
            'C:/Users/u10054206/OneDrive - NOS SGPS, S.A/Carreira/Portfolio/Python Projects/Scrapy/boardgames/boardgames.db')
cur = con.cursor()
query = f'SELECT id, name, price, url, date FROM jugamosotra WHERE id IN (%s) AND date="{date.today()}"' % ','.join('?' for i in games_id)
cur.execute(query, games_id)
df = cur.fetchall()

In [14]:
df = pd.DataFrame(df, columns=['id', 'name', 'price','url' ,'date'])

In [15]:
df

,id,name,price,url,date
0,300877.0,New York Zoo,31.49,https://jugamosotra.com/es/juegos/5388-new-yor...,2022-05-22
1,318084.0,Furnace + Promo,24.37,https://jugamosotra.com/es/juegos/6732-furnace...,2022-05-22
2,199966.0,Kingsburg Segunda Edición,44.95,https://jugamosotra.com/es/juegos/6471-kingsbu...,2022-05-22
3,264055.0,Draftosaurus: Marina,11.65,https://jugamosotra.com/es/juegos/5866-draftos...,2022-05-22


In [11]:
def send_mail():
    for i, game in df.iterrows():
        if game['price'] <= price_wanted:
            print(game['name'])
            print(game['price'])

            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                x = keyring.get_credential(service_name='mail_pess',username=None)
                user_mail = x.username
                pass_mail = x.password
                smtp.login(user_mail, pass_mail)

                msg = EmailMessage()
                msg['Subject'] = f"{game['name']} is at {game['price']}€! Now is your chance to buy!"
                msg['From'] = user_mail
                msg['To'] = user_mail
                msg.set_content(f"Filipe, This is the moment we have been waiting for.\nNow is your chance to pick up {game['name']}.\nDon't mess it up! Link here: {game['url']}")

                smtp.send_message(msg)
                print('Message sent! Check your e-mail account!')

In [12]:
send_mail()

Draftosaurus: Marina
11.65
Message sent! Check your e-mail account!
